# EEG BCI Competition 2a — Simple GNN (GCN) on per-channel features
This notebook trains a lightweight **Graph Convolutional Network** without PyTorch Geometric.

**Split is identical** to the CNN notebook (and your teammate): `random_state=42`, `test_size=0.2`, `stratify=y`.

We build a fixed graph over channels and use **bandpower features** per channel (Welch PSD) as node features.


In [31]:
# !pip install -U numpy pandas scikit-learn torch scipy mne


In [32]:
import os, glob
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from scipy.signal import welch
from mne.filter import filter_data



In [33]:
# =====================
# Config (EDIT ME)
# =====================
DATA_DIR = r"C:\Users\daiva\Desktop\Applied-Machine-Learning-Project\Dataset\BCI Competition 2a\Trials"
CSV_GLOB = os.path.join(DATA_DIR, "A??T_trials.csv")
SUBJECTS = None  # set e.g. ["A01T"] to match teammate's single-subject scope

TEST_SIZE = 0.2
SPLIT_SEED = 42
USE_STRATIFY = True

# Signal params (BCI Competition 2a commonly uses 250 Hz)
FS = 250
BANDS = [(1,4), (4,8), (8,13), (13,30), (30,40)]  # delta, theta, alpha, beta, gamma

BATCH_SIZE = 64
EPOCHS = 30
LR = 1e-3

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("DEVICE:", DEVICE)


DEVICE: cuda


In [34]:
# =====================
# Load preprocessed trials if available; otherwise recreate
# =====================
def load_trials_from_csv(csv_path: str):
    header = pd.read_csv(csv_path, nrows=0)
    eeg_cols = [c for c in header.columns if c.startswith('EEG')]
    usecols = ['Trial_ID', 'Label', 'Subject'] + eeg_cols
    df = pd.read_csv(csv_path, usecols=usecols)

    trial_ids = sorted(df['Trial_ID'].unique())
    n_times = (df[df['Trial_ID'] == trial_ids[0]].shape[0])
    n_channels = len(eeg_cols)

    X = np.zeros((len(trial_ids), n_channels, n_times), dtype=np.float32)
    y = np.zeros((len(trial_ids),), dtype=np.int64)
    trial_keys = []
    for i, tid in enumerate(trial_ids):
        trial = df[df['Trial_ID'] == tid]
        X[i] = trial[eeg_cols].to_numpy(dtype=np.float32).T
        y[i] = int(trial['Label'].iloc[0])
        subj = str(trial['Subject'].iloc[0])
        trial_keys.append(f"{subj}_{int(tid)}")
    return X, y, trial_keys, eeg_cols

if os.path.exists('preprocessed_trials.npz'):
    z = np.load('preprocessed_trials.npz', allow_pickle=True)
    X = z['X']
    y = z['y']
    keys = z['keys'].tolist()
    eeg_cols = z['eeg_cols'].tolist()
    label_values = z['label_values'].tolist()
    print('Loaded preprocessed_trials.npz', X.shape)
else:
    csv_files = sorted(glob.glob(CSV_GLOB))
    if not csv_files:
        raise FileNotFoundError(f"No CSV files matched {CSV_GLOB}. Check DATA_DIR.")
    Xs, ys, keys = [], [], []
    eeg_cols_ref = None
    for p in csv_files:
        Xp, yp, kp, eeg_cols_p = load_trials_from_csv(p)
        if SUBJECTS is not None:
            subj = kp[0].split('_')[0]
            if subj not in SUBJECTS:
                continue
        if eeg_cols_ref is None:
            eeg_cols_ref = eeg_cols_p
        elif eeg_cols_p != eeg_cols_ref:
            raise ValueError(f"EEG columns mismatch in {p}")
        Xs.append(Xp); ys.append(yp); keys.extend(kp)
    X = np.concatenate(Xs, axis=0)
    y_raw = np.concatenate(ys, axis=0)
    label_values = sorted(np.unique(y_raw).tolist())
    label_to_idx = {lab:i for i,lab in enumerate(label_values)}
    y = np.array([label_to_idx[int(v)] for v in y_raw], dtype=np.int64)
    eeg_cols = eeg_cols_ref
    np.savez('preprocessed_trials.npz', X=X, y=y, keys=np.array(keys), eeg_cols=np.array(eeg_cols), label_values=np.array(label_values))
    print('Saved preprocessed_trials.npz', X.shape)

n_trials, n_channels, n_times = X.shape
n_classes = len(np.unique(y))
print('Trials:', n_trials, 'Channels:', n_channels, 'Times:', n_times, 'Classes:', n_classes)


Saved preprocessed_trials.npz (2592, 22, 1000)
Trials: 2592 Channels: 22 Times: 1000 Classes: 4


In [ ]:
# ======= IDENTICAL to randomforestv2 preprocessing =======
from mne.filter import filter_data

SFREQ = 250.0
L_FREQ = 8.0
H_FREQ = 30.0

# X is (n_trials, n_channels, n_times)
X_filt = np.zeros_like(X, dtype=np.float64)

for i in range(X.shape[0]):
    X_filt[i] = filter_data(
        X[i].astype(np.float64),   # <-- THIS is the fix
        sfreq=SFREQ,
        l_freq=L_FREQ,
        h_freq=H_FREQ,
        verbose=False
    )

X = X_filt.astype(np.float32)  # back to float32 for torch
print("Filtered X:", X.shape, X.dtype)


In [ ]:
# =====================
# Split indices (reuse if present)
# =====================
if os.path.exists('split_indices_seed42.npz'):
    z = np.load('split_indices_seed42.npz', allow_pickle=True)
    train_idx, test_idx = z['train_idx'], z['test_idx']
    print('Loaded split_indices_seed42.npz')
else:
    idx = np.arange(len(y))
    strat = y if USE_STRATIFY else None
    train_idx, test_idx = train_test_split(idx, test_size=TEST_SIZE, random_state=SPLIT_SEED, stratify=strat)
    np.savez('split_indices_seed42.npz', train_idx=train_idx, test_idx=test_idx, keys=np.array(keys), y=y)
    print('Saved split_indices_seed42.npz')

print('Train:', len(train_idx), 'Test:', len(test_idx))


Loaded split_indices_seed42.npz
Train: 2073 Test: 519


In [ ]:
# =====================
# Node features: bandpower per channel per trial (Welch PSD)
# We'll cache to disk because this can take a few minutes.
# =====================
def bandpower_features(trial_ch_time: np.ndarray, fs: int, bands):
    # trial_ch_time: (C, T)
    feats = np.zeros((trial_ch_time.shape[0], len(bands)), dtype=np.float32)
    for c in range(trial_ch_time.shape[0]):
        f, Pxx = welch(trial_ch_time[c], fs=fs, nperseg=min(256, trial_ch_time.shape[1]))
        for bi, (lo, hi) in enumerate(bands):
            mask = (f >= lo) & (f < hi)
            # integrate PSD over band
            feats[c, bi] = np.trapz(Pxx[mask], f[mask]).astype(np.float32)
    # log transform to reduce skew
    return np.log(feats + 1e-10)

feat_cache = 'gnn_node_features_logbandpower.npz'
if os.path.exists(feat_cache):
    z = np.load(feat_cache)
    F = z['F']  # (N, C, B)
    print('Loaded', feat_cache, F.shape)
else:
    F = np.zeros((n_trials, n_channels, len(BANDS)), dtype=np.float32)
    for i in range(n_trials):
        F[i] = bandpower_features(X[i], FS, BANDS)
        if (i+1) % 100 == 0:
            print(f'Computed {i+1}/{n_trials}')
    np.savez(feat_cache, F=F)
    print('Saved', feat_cache)


Loaded gnn_node_features_logbandpower.npz (2592, 22, 5)


In [ ]:
# ===== Build trial-specific adjacency matrices =====

# AFTER bandpass, BEFORE corr_adj
FS = 250
X = X[:, :, int(2*FS):int(6*FS)]


def corr_adj(trial, thr=0.3):
    """
    trial: (C, T)
    Signed, sparse, normalized adjacency
    """
    A = np.corrcoef(trial)
    A = np.nan_to_num(A)

    # keep sign (DO NOT abs)
    A[np.abs(A) < thr] = 0.0

    # self-loops
    np.fill_diagonal(A, 1.0)

    # normalize by max abs value
    maxv = np.max(np.abs(A))
    if maxv > 0:
        A = A / maxv

    return A.astype(np.float32)



A_trials = np.stack([corr_adj(X[i], thr=0.5) for i in range(len(X))])
A_train = A_trials[train_idx]
A_test  = A_trials[test_idx]
print("Adjacency shape:", A_trials.shape)  # (N, C, C)


Adjacency shape: (2592, 22, 22)


In [ ]:
# Standardize node features using TRAIN only
F_train = F[train_idx]
mean = F_train.mean(axis=(0,1), keepdims=True)
std = F_train.std(axis=(0,1), keepdims=True) + 1e-8
Fn = (F - mean) / std

Fn_train = Fn[train_idx]
Fn_test  = Fn[test_idx]
y_train = y[train_idx]
y_test  = y[test_idx]

class_counts = np.bincount(y_train)
class_weights = class_counts.sum() / class_counts
class_weights = torch.tensor(class_weights, device=DEVICE, dtype=torch.float32)

crit = nn.CrossEntropyLoss(weight=class_weights)

print('Standardized node features.')


Standardized node features.


In [ ]:
# =====================
# Build fixed adjacency over channels
# For simplicity we use a fully-connected graph + self loops.
# =====================
C = n_channels
A = np.ones((C, C), dtype=np.float32)
np.fill_diagonal(A, 1.0)

# Normalize: A_hat = D^{-1/2} A D^{-1/2}
D = np.sum(A, axis=1)
D_inv_sqrt = np.diag(1.0 / np.sqrt(D + 1e-8))
A_hat = D_inv_sqrt @ A @ D_inv_sqrt
A_hat = torch.from_numpy(A_hat).to(DEVICE)

print('Adjacency ready:', A_hat.shape)


Adjacency ready: torch.Size([22, 22])


In [ ]:
class GraphDataset(Dataset):
    def __init__(self, F, A, y):
        self.F = torch.from_numpy(F).float()
        self.A = torch.from_numpy(A).float()
        self.y = torch.from_numpy(y).long()

    def __len__(self):
        return len(self.y)

    def __getitem__(self, i):
        return self.F[i], self.A[i], self.y[i]

train_loader = DataLoader(
    GraphDataset(Fn_train, A_train, y_train),
    batch_size=BATCH_SIZE,
    shuffle=True
)

test_loader = DataLoader(
    GraphDataset(Fn_test, A_test, y_test),
    batch_size=BATCH_SIZE,
    shuffle=False
)


In [ ]:
class BigGCN(nn.Module):
    def __init__(self, in_feats, hidden, n_classes):
        super().__init__()

        self.fc_in = nn.Linear(in_feats, hidden)

        self.gcn1 = nn.Linear(hidden, hidden)
        self.gcn2 = nn.Linear(hidden, hidden)
        self.gcn3 = nn.Linear(hidden, hidden)

        self.norm1 = nn.LayerNorm(hidden)
        self.norm2 = nn.LayerNorm(hidden)
        self.norm3 = nn.LayerNorm(hidden)

        self.attn = nn.Sequential(
            nn.Linear(hidden, 1),
            nn.Softmax(dim=1)
        )

        self.cls = nn.Linear(hidden, n_classes)
        self.act = nn.ReLU()

    def gcn(self, X, A, layer, norm):
        H = layer(X)
        H = torch.bmm(A, H)
        return self.act(norm(H))

    def forward(self, X, A):
        # X: (B, C, F), A: (B, C, C)
        H = self.fc_in(X)

        H1 = self.gcn(H, A, self.gcn1, self.norm1)
        H2 = self.gcn(H1 + H, A, self.gcn2, self.norm2)
        H3 = self.gcn(H2 + H1, A, self.gcn3, self.norm3)

        # attention pooling over nodes
        w = self.attn(H3)              # (B, C, 1)
        g = (H3 * w).sum(dim=1)        # (B, hidden)

        return self.cls(g)


model = BigGCN(
    in_feats=Fn.shape[-1],
    hidden=128,
    n_classes=n_classes
).to(DEVICE)

opt = torch.optim.Adam(model.parameters(), lr=LR)
crit = nn.CrossEntropyLoss()
print(model)


BigGCN(
  (fc_in): Linear(in_features=5, out_features=128, bias=True)
  (gcn1): Linear(in_features=128, out_features=128, bias=True)
  (gcn2): Linear(in_features=128, out_features=128, bias=True)
  (gcn3): Linear(in_features=128, out_features=128, bias=True)
  (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
  (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
  (norm3): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
  (attn): Sequential(
    (0): Linear(in_features=128, out_features=1, bias=True)
    (1): Softmax(dim=1)
  )
  (cls): Linear(in_features=128, out_features=4, bias=True)
  (act): ReLU()
)


In [ ]:
# ===== Evaluation helper (GNN version) =====
def evaluate(model, loader):
    model.eval()
    ys, ps = [], []
    with torch.no_grad():
        for xb, Ab, yb in loader:
            xb = xb.to(DEVICE)
            Ab = Ab.to(DEVICE)
            logits = model(xb, Ab)
            pred = logits.argmax(dim=1).cpu().numpy()
            ys.append(yb.numpy())
            ps.append(pred)
    y_true = np.concatenate(ys)
    y_pred = np.concatenate(ps)
    return y_true, y_pred

def drop_edges(A, p=0.2):
    # A: (B, C, C)
    mask = (torch.rand_like(A) > p).float()
    return A * mask

# ===== Training loop =====
for epoch in range(1, EPOCHS + 1):
    model.train()
    losses = []

    for xb, Ab, yb in train_loader:
        xb = xb.to(DEVICE)
        Ab = Ab.to(DEVICE)
        yb = yb.to(DEVICE)

        Ab = drop_edges(Ab, p=0.2)  # TRAIN ONLY
        opt.zero_grad()

        logits = model(xb, Ab)
        loss = crit(logits, yb)

        loss.backward()
        opt.step()

        losses.append(loss.item())

    # ----- Evaluate -----
    ytr, ptr = evaluate(model, train_loader)
    yte, pte = evaluate(model, test_loader)

    tr_acc = accuracy_score(ytr, ptr)
    te_acc = accuracy_score(yte, pte)

    print(
        f"Epoch {epoch:02d} | loss {np.mean(losses):.4f} | "
        f"train acc {tr_acc:.3f} | test acc {te_acc:.3f}"
    )


# ===== Final evaluation =====
y_true, y_pred = evaluate(model, test_loader)

print("\n=== GNN Test Set Report ===")
print("Accuracy:", accuracy_score(y_true, y_pred))
print(classification_report(y_true, y_pred, target_names=[str(v) for v in label_values]))
print("Confusion matrix:\n", confusion_matrix(y_true, y_pred))


Epoch 01 | loss 1.3980 | train acc 0.315 | test acc 0.270
Epoch 02 | loss 1.3650 | train acc 0.317 | test acc 0.276
Epoch 03 | loss 1.3651 | train acc 0.321 | test acc 0.279
Epoch 04 | loss 1.3571 | train acc 0.320 | test acc 0.285
Epoch 05 | loss 1.3530 | train acc 0.344 | test acc 0.276
Epoch 06 | loss 1.3430 | train acc 0.333 | test acc 0.270
Epoch 07 | loss 1.3469 | train acc 0.316 | test acc 0.285
Epoch 08 | loss 1.3467 | train acc 0.326 | test acc 0.291
Epoch 09 | loss 1.3439 | train acc 0.349 | test acc 0.272
Epoch 10 | loss 1.3409 | train acc 0.345 | test acc 0.272
Epoch 11 | loss 1.3381 | train acc 0.338 | test acc 0.276
Epoch 12 | loss 1.3349 | train acc 0.342 | test acc 0.295
Epoch 13 | loss 1.3310 | train acc 0.347 | test acc 0.270
Epoch 14 | loss 1.3366 | train acc 0.357 | test acc 0.279
Epoch 15 | loss 1.3288 | train acc 0.360 | test acc 0.293
Epoch 16 | loss 1.3324 | train acc 0.347 | test acc 0.287
Epoch 17 | loss 1.3266 | train acc 0.354 | test acc 0.277
Epoch 18 | los

In [ ]:
# Save model + feature standardization
torch.save({
    'model_state_dict': model.state_dict(),
    'label_values': label_values,
    'eeg_cols': eeg_cols,
    'bands': BANDS,
    'fs': FS,
    'feat_mean': mean.astype(np.float32),
    'feat_std': std.astype(np.float32),
}, 'gnn_model_seed42.pt')
print('Saved gnn_model_seed42.pt')


Saved gnn_model_seed42.pt


In [ ]:
import pandas as pd
from sklearn.metrics import f1_score
from datetime import datetime
import os

# ===== CONFIG =====
RESULTS_CSV = "experiment_results.csv"
MODEL_TYPE = "GNN"   # change to "GNN" in the GNN notebook
SUBJECT = "A02T"     # update dynamically if looping subjects
SEED = 42
FS = 250
BANDPASS = "8-30"
ZSCORE = True        # False if you disable it
TIME_WINDOW = "full" # or "2-6s"

# ===== METRICS =====
test_acc = accuracy_score(y_true, y_pred)
macro_f1 = f1_score(y_true, y_pred, average="macro")

row = {
    "timestamp": datetime.now().isoformat(timespec="seconds"),
    "model_type": MODEL_TYPE,
    "subject": SUBJECT,
    "seed": SEED,
    "fs": FS,
    "bandpass": BANDPASS,
    "zscore": ZSCORE,
    "time_window": TIME_WINDOW,
    "n_trials_train": len(train_idx),
    "n_trials_test": len(test_idx),
    "test_accuracy": test_acc,
    "macro_f1": macro_f1,
}

df_row = pd.DataFrame([row])

# ===== APPEND OR CREATE =====
if os.path.exists(RESULTS_CSV):
    df_row.to_csv(RESULTS_CSV, mode="a", header=False, index=False)
else:
    df_row.to_csv(RESULTS_CSV, index=False)

print("Logged results to", RESULTS_CSV)
df_row


Logged results to experiment_results.csv


,timestamp,model_type,subject,seed,fs,bandpass,zscore,time_window,n_trials_train,n_trials_test,test_accuracy,macro_f1
0,2025-12-17T15:16:52,GNN,A01T,42,250,8-30,True,full,2073,519,0.254335,0.232277
